<a href="https://colab.research.google.com/github/YAMAZAKIAkiraRK/GCAnet/blob/main/DataAugTest4_contrast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/GCA/

/content/drive/MyDrive/Colab Notebooks/GCA


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3

import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn.preprocessing as sp
import cv2
from sklearn.model_selection import train_test_split
import preprocess as pp
import Grad_Cam as gc
from PIL import ImageOps, Image

In [ ]:
#訓練データの読み込み
p_dir = "YU/grating image/202207/GC2022"
c_dir = os.listdir(p_dir)
i = 0
data=[]
label=[]
label_name = []

for d in c_dir:
    label_name.append(d)
    d = os.path.join(p_dir, d)
    pp.append_data(d, data, label, i)
    i += 1
label = np.array(label).reshape(-1,1)
#ラベルをone_hot_encoding
enc = sp.OneHotEncoder(categories="auto", sparse_output=False, dtype=np.float32)
label_enc = enc.fit_transform(label)


#テストデータの読み込み
p_dir = "YU/grating image/202207/test data"
c_dir = os.listdir(p_dir)
i = 0
test_data=[]
test_label=[]
test_label_name = []
for d in c_dir:
    test_label_name.append(d)
    d = os.path.join(p_dir, d)
    pp.append_data(d, test_data, test_label, i)
    i += 1

In [ ]:
#前処理-アスペクト比は気にせず299,299に縮小
#訓練データ
shrink_data = np.empty((len(data),299,299,3))
for i in range(len(data)):
    shrink_data[i]=cv2.resize(data[i], dsize=(299, 299))
shrink_data = np.array(shrink_data)

#テストデータ
shrink_test_data = np.empty((len(test_data),299,299,3))
for i in range(len(test_data)):
    shrink_test_data[i]=cv2.resize(test_data[i], dsize=(299, 299))
shrink_test_data = np.array(shrink_test_data)

In [82]:
#各種autocontrastでデータを複製
ac_datas = []
for i in range(4):
  ac_data = []
  for r in range(len(shrink_data)):
    ac = shrink_data[r].astype(np.uint8)
    ac = Image.fromarray(ac)
    cutoff = i
    autocon = ImageOps.autocontrast(ac, cutoff)
    ac_data.append(np.array(autocon).astype(float))
  ac_datas.append(np.array(ac_data))

#テストデータも
ac_tests = []
for i in range(4):
  ac_test = []
  for r in range(len(shrink_test_data)):
    ac = shrink_test_data[r].astype(np.uint8)
    ac = Image.fromarray(ac)
    cutoff = i
    autocon = ImageOps.autocontrast(ac, cutoff)
    ac_test.append(np.array(autocon).astype(float))
  ac_tests.append(np.array(ac_test))

In [83]:
ac_data = []
for r in range(len(shrink_data)):
    ac = shrink_data[r].astype(float)
    ac = ac-30
    ac[ac<0]=0
    ac = ac*254/225
    ac_data.append(np.array(ac))
ac_datas.append(np.array(ac_data))

In [84]:
ac_test = []
for r in range(len(shrink_test_data)):
  ac = shrink_test_data[r].astype(float)
  ac = ac-30
  ac[ac<0]=0
  ac = ac*254/225
#  ac = ac.astype(np.uint8)
  ac_test.append(np.array(ac))
ac_tests.append(np.array(ac_test))

In [ ]:
ac_tests[0].shape

(41, 299, 299, 3)

In [87]:
#オートコントラストのパラメータ別
def ACcomparison(datas, label, test_data, test_label):
    models=[]
    histories=[]
    preds=[]
    for data in datas:
        #GoogLeNetを読み込み、出力層の数を6に変更
        base_model = InceptionV3(weights="imagenet", input_shape=(299, 299,3) ,include_top=False)
        x = base_model.output
        x = keras.layers.GlobalAveragePooling2D()(x)
        predictions = Dense(6, activation="softmax")(x)
        model = Model(inputs=base_model.input, outputs=predictions)
        #モデルのコンパイル
        for layer in base_model.layers:
            layer.trainable = False
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

        train_data, val_data, train_label, val_label = train_test_split(data, label, random_state=1, stratify = label)
        train_data /= 255
        val_data /= 255
        early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor="val_loss", min_delta=0.001, restore_best_weights=True, verbose=1)
        history = model.fit(train_data, train_label, batch_size=4,
                            steps_per_epoch = len(train_data)/4,
                            validation_data = (val_data, val_label),
                            epochs=120, verbose=1, callbacks=[early_stopping])

        models.append(model)
        histories.append(history)
        prediction = np.argmax(model.predict(test_data[i]),axis=1)
        preds.append(prediction)

    return models, histories, preds

In [88]:
models, histories, predictions = ACcomparison(
    datas=ac_datas, label=label_enc, test_data=ac_tests, test_label=test_label
)

Epoch 1/120
10/10 [==============================] - 7s 230ms/step - loss: 2.0487 - accuracy: 0.0976 - val_loss: 1.7727 - val_accuracy: 0.0714
Epoch 2/120
10/10 [==============================] - 1s 65ms/step - loss: 1.7183 - accuracy: 0.1707 - val_loss: 1.5501 - val_accuracy: 0.3571
Epoch 3/120
10/10 [==============================] - 1s 55ms/step - loss: 1.5087 - accuracy: 0.4878 - val_loss: 1.4351 - val_accuracy: 0.5714
Epoch 4/120
10/10 [==============================] - 1s 62ms/step - loss: 1.3901 - accuracy: 0.5610 - val_loss: 1.3542 - val_accuracy: 0.5000
Epoch 5/120
10/10 [==============================] - 1s 69ms/step - loss: 1.3150 - accuracy: 0.5122 - val_loss: 1.3023 - val_accuracy: 0.5000
Epoch 6/120
10/10 [==============================] - 1s 65ms/step - loss: 1.2413 - accuracy: 0.5854 - val_loss: 1.2621 - val_accuracy: 0.6429
Epoch 7/120
10/10 [==============================] - 1s 72ms/step - loss: 1.1834 - accuracy: 0.6341 - val_loss: 1.2207 - val_accuracy: 0.5714
Epoch

In [89]:
loss = []
for i in range(5):
  loss.append(histories[i].history["val_loss"][-1])

In [90]:
loss

[0.4015568196773529,
 0.5032914876937866,
 0.395497590303421,
 0.5483052134513855,
 0.3468170762062073]

In [91]:
preds_label_name = []
for predict in predictions:
  pred_label_name = []
  for idx in predict:
    pred_label_name.append(label_name[idx])
  pred_label_name = np.array(pred_label_name)
  preds_label_name.append(pred_label_name)

In [97]:
predictions_origin = []
for i in range(5):
  pred = np.argmax(models[i].predict(shrink_test_data), axis=1)
  predictions_origin.append(pred)

2/2 [==============================] - 0s 106ms/step


In [98]:
origin_preds_label_name = []
for predict in predictions_origin:
  pred_label_name = []
  for idx in predict:
    pred_label_name.append(label_name[idx])
  pred_label_name = np.array(pred_label_name)
  origin_preds_label_name.append(pred_label_name)

In [92]:
correct_label_name = []
for idx in test_label:
  correct_label_name.append(test_label_name[idx])

In [93]:
accuracies=[]
for i in range(len(preds_label_name)):
  accuracies.append(sum(preds_label_name[i] == correct_label_name))

In [99]:
origin_accuracies=[]
for i in range(len(origin_preds_label_name)):
  origin_accuracies.append(sum(origin_preds_label_name[i] == correct_label_name))
origin_accuracies

[12, 12, 7, 7, 16]

In [94]:
accuracies

[12, 12, 7, 7, 14]

In [ ]:
NDApred_label_name = []
for idx in NDAprediction:
  NDApred_label_name.append(label_name[idx])
NDApred_label_name = np.array(NDApred_label_name)

In [ ]:
NDAaccuracy = sum(NDApred_label_name == correct_label_name)
NDAaccuracy

27

In [ ]:
model = models[6]
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_564 (Conv2D)            (None, 149, 149, 32  864         ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_564 (Batch  (None, 149, 149, 32  96         ['conv2d_564[0][0]']             
 Normalization)                 )                                                           

In [ ]:
model.layers[-1].activation = None
img_path = "shrink/shrink_testdata/shrink_testdata16.jpg"
img_array = gc.get_img_array(img_path)
last_conv_layer_name = "conv2d_657"
heatmap = gc.make_gradcam_heatmap(img_array, model, last_conv_layer_name)
gc.save_and_display_gradcam(img_path, heatmap, cam_path="cam6.jpg")

In [ ]:
import pandas as pd
df_shrink = pd.DataFrame()
df_shrink["Predict_Class"] = pred_label_name
df_shrink["True_Class"] = correct_label_name
print(pd.crosstab(df_shrink["Predict_Class"], df_shrink["True_Class"]))
print("サンプル数：", len(df_shrink["Predict_Class"]))
print("正解数：", sum(df_shrink["Predict_Class"] == df_shrink["True_Class"]))

In [ ]:
model.layers[-1].activation = None
img_path = "shrink/shrink_testdata/shrink_testdata16.jpg"
img_array = gc.get_img_array(img_path)
last_conv_layer_name = "conv2d_657"
heatmap = gc.make_gradcam_heatmap(img_array, model, last_conv_layer_name)
gc.save_and_display_gradcam(img_path, heatmap, cam_path="cam6.jpg")